# fBPM Deep Learning analysis
Script that analyzes multiple datasets with deep learning after eachother, and saves the analysis of a .txt file in a folder with the same name.

### Import Libraries and functions

In [1]:
import os
import random
import scipy
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import keras
from keras import models
from keras import layers
from random import sample
from numpy.random import shuffle
import os, shutil
#base_dir = '/Users/s136769/stack/MBx Python/MSD script'
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import sys
import sklearn as sk
import tensorflow.keras
import tensorflow as tf
from lmfit import Model, Parameters

In [2]:
def import_data(filename):
    nparray=np.array(pd.read_csv(filename, header=None))
    return nparray

def obtain_xy_per_particle(data_txt):
    num_part = data_txt.shape[0]
    x_coord = np.zeros((num_part,int(data_txt.shape[1]/2-1)))
    y_coord = np.zeros((num_part,int(data_txt.shape[1]/2-1)))
    for i in range(num_part):
        x0 = data_txt[i,0]
        y0 = data_txt[i,1]
        x_coord[i,:] = data_txt[i,2::2]+x0
        y_coord[i,:] = data_txt[i,3::2]+y0
    
    return [x_coord, y_coord]

def X_test_from_timeseries(x,y,N_timesteps):
    X_train=np.zeros((x.shape[0]-N_timesteps+1, N_timesteps, 2))
    for i in range(x.shape[0]-N_timesteps+1):
        X_train[i,:,0]=x[i:i+N_timesteps]
        X_train[i,:,1]=y[i:i+N_timesteps]
        
    return X_train

def obtain_state_per_pos(state_per_avg, measurementwindow):
    y=np.zeros(state_per_avg.shape[0]+measurementwindow-1)
    for i in range(y.shape[0]):
        if i<measurementwindow-1:
            y[i]=np.mean(state_per_avg[:i+1])
        elif i>state_per_avg.shape[0]-1:
            y[i]=np.mean(state_per_avg[i-measurementwindow+1:])
        else:
            y[i]=np.mean(state_per_avg[i-measurementwindow+1:i+1])
    y=np.round(y)
    return y

def smoother_unev(a, smooth): #Watch out!! works only OK when inputting a uneven smooth prm -> otherwise avg doesnt work
    ret=np.zeros(a.shape[0])
    smooth_prm=(smooth-1)/2
    for i in range(a.shape[0]):
        if i+1<=smooth_prm:
            ret[i]=np.mean(a[:i+1+int(smooth_prm)])
        elif i+1>a.shape[0]-smooth_prm:
            ret[i]=np.mean(a[i-int(smooth_prm):a.shape[0]])
        else:
            ret[i]=np.mean(a[i-int(smooth_prm):i+1+int(smooth_prm)])
    ret=np.round(ret)
    return ret

def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

def diff_coef_over_time(x,y,measurementwindow,no_MSD_val, FrameRate):
    DtraceWindow=np.zeros((no_MSD_val,x.shape[0]-measurementwindow+1))
    weight = np.zeros(no_MSD_val)
    for dt in range(no_MSD_val):
        SDtrace = np.square(x[dt+1:]-x[:-dt-1]) + np.square(y[dt+1:]-y[:-dt-1])
        SDtraceWindow = moving_average(SDtrace, measurementwindow-dt-1)
        
        DtraceWindow[dt,:]=SDtraceWindow*FrameRate/(4*(dt+1)) #(no_MSD, no_x_val)
        Vrel = (dt+1)*(2*np.square(dt+1)+1)/(measurementwindow-dt)
        weight[dt]=1/Vrel
        
    weight_norm = weight/np.sum(weight)
    D_tot=np.sum(np.multiply(DtraceWindow.T, weight_norm).T, axis=0)
    return D_tot

def expand_D_val(D, measurementwindow):
    D_new=np.zeros(D.shape[0]+measurementwindow-1)
    for i in range(D_new.shape[0]):
        if i<measurementwindow-1:
            D_new[i]=np.mean(D[:i+1])
        elif i>D.shape[0]-1:
            D_new[i]=np.mean(D[i-measurementwindow+1:])
        else:
            D_new[i]=D[i-measurementwindow+1]
    return D_new

def obtain_pred_state2(x_test, y_test, state2_test, measurementwindow, norm, model, smooth):
    i=0
    save=0
    pred_state2=np.zeros(state2_test.shape[0])
    while i<state2_test.shape[0]-1:
        if state2_test[i]>0:
            begin_point=i
            
        while state2_test[i]>0 and i<state2_test.shape[0]-1:
            i=i+1
            save=1
        
        if save==1:
            if state2_test[begin_point:i].shape[0]<=measurementwindow:
                pred_state2[begin_point:i]=1
                save=0
            else:            
                x_arr=x_test[begin_point:i]
                y_arr=y_test[begin_point:i]

                X_test=X_test_from_timeseries(x_arr,y_arr,measurementwindow)
                # Normalize in the same way as the training data
                for k in range(X_test.shape[0]):
                    X_test[k,:,:]-=X_test[k,0,:] # minus the first coordinate to start the trajectories in the same spot.
            
                X_test=X_test/norm #normalize by the bound radius of the first state.
                
                Y_pred=model.predict(X_test) #You could round these predictions -> or set different probability threshold.
                pred_state2[begin_point:i]=smoother_unev(obtain_state_per_pos(Y_pred, measurementwindow)+1, smooth)
                save=0    
        i=i+1
    pred_state2[-1]=pred_state2[-2]
    return pred_state2

# april 2021 -> addition of option to downscale 60 fps to 30 fps.
def downscale_FR60_to_FR30(x_per_part, y_per_part):
    x_per_part30 = x_per_part[:,np.arange(0,x_per_part.shape[1]-1,2)]
    y_per_part30 = y_per_part[:,np.arange(0,y_per_part.shape[1]-1,2)]
    N_frames = x_per_part30.shape[1]
    FrameRate=30
    print('DOWNSCALED TO 30 FPS: The number of frames per particle is equal to %1.0f' %N_frames + ', which corresponds to %1.0f seconds.' %np.floor(N_frames/FrameRate))
    return [x_per_part30, y_per_part30, N_frames, FrameRate]

# Reduce the traces to 1 frame per second -> used in the find_mislocalizations function to speed up the search.
def reduce_to_1frame_ps(x, FrameRate):
    N_frames = x.shape[1]
    x_1FR = x[:,np.linspace(0,N_frames-FrameRate, int(N_frames/FrameRate)).astype(int)]
    return x_1FR

# Filter out the particles that show mislocalizations -> which means that two particles move in the vicinity of each other 
# -> consequently, the localization algorithm cannot distinguish the particles and the 
def find_mislocalizations(x_per_part, y_per_part, FrameRate):
    # Round up to a micrometer -> afterwards, check whether particles have same coordinate value in rounded micrometers.
    x_per_part_round = np.round(x_per_part)
    y_per_part_round = np.round(y_per_part)
    # Look at the coordinate every second, rather than every frame.
    x_round_short = reduce_to_1frame_ps(x_per_part_round, FrameRate)
    y_round_short = reduce_to_1frame_ps(y_per_part_round, FrameRate)
    
    # set the threshold for number of coordinates two traces must have in common on same timestep to be flagged as mislocalized.
    # Currently this is 10 -> hence the 1/10.
    N_overlap_threshold = np.round(x_round_short.shape[1]/10)
    Mislocalization = np.array([])
    for n_part in range(x_round_short.shape[0]):
        #Record wether different particles have the same trace (same coordinates at same time)
        x_same_coord = x_round_short==x_round_short[n_part,:]
        y_same_coord = y_round_short==y_round_short[n_part,:]
        xy_overlap = np.logical_and(x_same_coord, y_same_coord)
        N_overlap_per_particle = np.sum(xy_overlap,axis=1)
        index_misl = np.where(N_overlap_per_particle>N_overlap_threshold)[0]
        # Need to filter out the particle n_part itself.
        if index_misl.shape[0]>1:
            index_misl = np.delete(index_misl,np.where(index_misl==n_part)[0])
            Mislocalization = np.append(Mislocalization, index_misl)
    #sort and remove duplicates from the mislocalization array    
    Mislocalization_final = np.unique(Mislocalization)
    return Mislocalization_final

def MSD_from_xy(x,y,windowsize,no_MSD_val):
    DtraceWindow = np.zeros((no_MSD_val, x.size-windowsize+1))
    for dt in range(no_MSD_val):
        SDtrace=np.square(x[1+dt:]-x[:-dt-1])+np.square(y[1+dt:]-y[:-dt-1]) #squared displacement     
        SDtraceWindow=moving_average(SDtrace,windowsize-dt-1)
        DtraceWindow[dt,:]=SDtraceWindow #;%*FrameRate/(4*dt); %was SDtraceWindow/(4*dt)
    return DtraceWindow

def find_freely_diffusing_particles(act01, x_per_part, y_per_part, FrameRate):
    Nframes = x_per_part.shape[1]
    Npart = x_per_part.shape[0]
    maxdt = 10
    time = np.arange(0,maxdt+1,1)/FrameRate
    # loop over all the particles
    DC_part = np.zeros(Npart)
    for i in range(Npart):
        x = x_per_part[i,:]
        y = y_per_part[i,:]
        MSD = np.zeros(maxdt+1)
        for dt in range(maxdt):
            SD = np.square(x[1+dt:]-x[:-dt-1])+np.square(y[1+dt:]-y[:-dt-1])
            MSD[dt+1] = np.mean(SD)
        # apply a fit to the MSD values
        # use linear regression for fast result.
        reg = LinearRegression(fit_intercept=True).fit(time.reshape(-1,1),MSD.reshape(-1,1))
        # reg.intercept_ -> if you want to obtain the intercept.
        DC_part[i] = reg.coef_[0][0]/4
        
    DC_threshold_freely_diff = np.mean(DC_part)
    freely_boolean = np.logical_and(DC_part>DC_threshold_freely_diff, act01==0)
    freely_ind = np.where(freely_boolean)[0]
    return freely_ind
    
# find stuck particles similarly to Alissa script.
# Fit the MSD of the whole trace -> determine diffusion coefficient -> particle=stuck, if DC<threshold.
def find_stuck_particles(x_per_part, y_per_part, FrameRate, MVbound_threshold):
    Nframes = x_per_part.shape[1]
    Npart = x_per_part.shape[0]
    maxdt = 10
    time = np.arange(0,maxdt+1,1)/FrameRate
    # loop over all the particles
    DC_part = np.zeros(Npart)
    for i in range(Npart):
        x = x_per_part[i,:]
        y = y_per_part[i,:]
        MSD = np.zeros(maxdt+1)
        for dt in range(maxdt):
            SD = np.square(x[1+dt:]-x[:-dt-1])+np.square(y[1+dt:]-y[:-dt-1])
            MSD[dt+1] = np.mean(SD)
        # apply a fit to the MSD values
        # use linear regression for fast result.
        reg = LinearRegression(fit_intercept=True).fit(time.reshape(-1,1),MSD.reshape(-1,1))
        # reg.intercept_ -> if you want to obtain the intercept.
        DC_part[i] = reg.coef_[0][0]/4

    # Obtain the indexes from the particles for which the avg. diffusion coefficient is smaller than the threshold.
    stuck_ind = np.where(DC_part<MVbound_threshold)[0]
    return stuck_ind
    
# Function for obtaining the mean and standard deviation of fitting a poisson distribution.
def obtain_poisson_fit_stat(events_per_particle):
    N_particles = events_per_particle.shape[0]
    # Using MLE to fit a poisson distribution -> parameter lambda is equal to the mean of the observations, the variance of a poisson distribution is equal to lambda as well.
    prm_pois = np.mean(events_per_particle)
    std_pois = np.sqrt(prm_pois)
    SE_pois = std_pois/np.sqrt(N_particles)
    return [prm_pois, std_pois, SE_pois]

# Function for obtaining the mean and standard deviation of fitting a normal distribution.
def obtain_normal_fit_stat(events_per_particle):
    N_particles = events_per_particle.shape[0]
    # Using MLE to fit a normal distribution -> prms mean&std are simply the mean and std of the observations.
    mean_normal = np.mean(events_per_particle) 
    std_normal = np.std(events_per_particle)
    SE_normal = std_normal/np.sqrt(N_particles)
    return [mean_normal, std_normal, SE_normal]
    
# Calculate activity function for the filtering.
def calc_activity_of_particles(pred_state_all):
    analyzed_particles = pred_state_all.shape[0]
    act01 = np.zeros(analyzed_particles)
    act12 = np.zeros(analyzed_particles)
    acttot = np.zeros(analyzed_particles)
    for i in range(analyzed_particles):
        [activity01, activity12, activitytotal] = calculate_activity_per_particle(pred_state_all[i,:])
        act01[i] = activity01
        act12[i] = activity12
        acttot[i] = activitytotal
    return [act01, act12, acttot]

## Filter out particles that have an unusually large activity -> same removal method as Alissa uses. 

# Iterate until no particles are filtered out anymore.
# Apply the filtering for both the 01 activity and the 12 activity?
# Or apply the function on both the 01 activity and the 12 activity seperately.
def filter_large_activity(events_per_particle, stuck_particle_ind,distribution_fit,sigma):
    # Input: both np.arrays
    
    # Need to take out the stuck particles from the analysis, but keep the array the same, to obtain the correct particle index.
    # create boolean array to indicate particles to be removed.
    particles_stuck = np.zeros(events_per_particle.shape[0])
    particles_stuck[stuck_particle_ind]=1
    particles_to_remove = np.zeros(events_per_particle.shape[0])
    stop_prm = 0
    
    if distribution_fit=='poisson':
        print('The events are filtered using the Poisson distribution.')
        while stop_prm!=1: 
            # calculate number of particles removed in previous cycle.
            N_removed_old = np.sum(particles_to_remove)
            # Remove the stuck particles & filtered particles and apply filtering again.
            particles_to_analyze = events_per_particle[np.logical_and(particles_stuck==0, particles_to_remove==0)]
            
            if particles_to_analyze.shape[0]>0:
                # Fit poisson and calculate the mean, std and SE of the indicated particles.
                [events_mean, events_std, events_SE] = obtain_poisson_fit_stat(particles_to_analyze)
                particles_to_remove = events_per_particle>events_mean+sigma*events_std

            if np.sum(particles_to_remove)==N_removed_old or particles_to_analyze.shape[0]==0:
                # no new particles are removed -> stop the while loop.
                stop_prm=1
            
    elif distribution_fit=='normal':
        print('The events are filtered using the Normal distribution.') 
        while stop_prm!=1: 
            # calculate number of particles removed in previous cycle.
            N_removed_old = np.sum(particles_to_remove)
            # Remove the stuck particles & filtered particles and apply filtering again.
            particles_to_analyze = events_per_particle[np.logical_and(particles_stuck==0, particles_to_remove==0)]
            
            if particles_to_analyze.shape[0]>0:
                # Fit poisson and calculate the mean, std and SE of the indicated particles.
                [events_mean, events_std, events_SE] = obtain_normal_fit_stat(particles_to_analyze)
                particles_to_remove = events_per_particle>events_mean+sigma*events_std
                
            if np.sum(particles_to_remove)==N_removed_old or particles_to_analyze.shape[0]==0:
                # no new particles are removed -> stop the while loop.
                stop_prm=1
    else:
        print('The indicated distribution_fit is not a valid input. Input options: poisson or normal.')
    
    filtered_particles = np.where(particles_to_remove)[0]
    return filtered_particles

def plot_diff_coef_hist(diff_coef_all, particle_index, incl_or_excl,upper_threshold,save, name):
    diff_coef_all_arr = np.array([])
    for i in range(diff_coef_all.shape[0]):
        if (i in particle_index and incl_or_excl==0) or (i not in particle_index and incl_or_excl==1):
            diff_coef_all_arr = np.append(diff_coef_all_arr, diff_coef_all[i])
    fig, ax = plt.subplots(1,1, constrained_layout=True)
    plt.hist(diff_coef_all_arr[diff_coef_all_arr<upper_threshold], bins=200)
    #plt.title("Diffusion coefficient distribution of selected particles")
    plt.rc('font', size=14)          # controls default text sizes
    plt.rc('axes', titlesize=14)     # fontsize of the axes title
    plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
    plt.rc('legend', fontsize=14)    # legend fontsize
    plt.rc('figure', titlesize=16)  # fontsize of the figure title
    plt.xlim(0,upper_threshold)
    plt.ylim(0,100000)
    #ax.set_yscale('linear')
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    ax.yaxis.major.formatter._useMathText = True
    ax.set_xlabel("D ($\u03BCm^2/s$)")
    plt.xticks([0,0.02,0.04,0.06,0.08,0.10],['0','0.02','0.04','0.06','0.08','0.10'])
    plt.yticks([0,20000,40000,60000,80000,100000])
    ax.set_ylabel("Counts")
    
    if save==1:
        plt.savefig(name+'\diffusion_all')
    plt.show()
    
def plot_diff_coef_hist_per_pred_state(diff_coef_all, pred_state_all, particle_index, incl_or_excl, N_val_D, upper_threshold,save,name):
    diff_coef_unb = np.array([])
    diff_coef_b1 = np.array([])
    diff_coef_b2 = np.array([])
    for i in range(pred_state_all.shape[0]):
        if (i in particle_index and incl_or_excl==0) or (i not in particle_index and incl_or_excl==1):
            diff_coef_unb = np.append(diff_coef_unb, diff_coef_all[i][pred_state_all[i][np.floor(N_val_D/2).astype(int)-1:-np.floor(N_val_D/2).astype(int)]==0])
            diff_coef_b1 = np.append(diff_coef_b1, diff_coef_all[i][pred_state_all[i][np.floor(N_val_D/2).astype(int)-1:-np.floor(N_val_D/2).astype(int)]==1])
            diff_coef_b2 = np.append(diff_coef_b2, diff_coef_all[i][pred_state_all[i][np.floor(N_val_D/2).astype(int)-1:-np.floor(N_val_D/2).astype(int)]==2])
    
    fig, ax = plt.subplots(1,1,constrained_layout=True)
    plt.hist(diff_coef_unb[diff_coef_unb<upper_threshold], bins=200, color='tab:blue', alpha=0.5, label='unbound')
    plt.hist(diff_coef_b1[diff_coef_b1<upper_threshold], bins=200, color='green', alpha=0.5, label='single bound')
    plt.hist(diff_coef_b2[diff_coef_b2<upper_threshold], bins=200, color='red', alpha=0.5, label='double bound')
    #plt.title("Seperate diffusion coefficient distributions of selected particles")
    plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
    plt.rc('font', size=14)          # controls default text sizes
    plt.rc('axes', titlesize=14)     # fontsize of the axes title
    plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
    plt.rc('legend', fontsize=14)    # legend fontsize
    plt.rc('figure', titlesize=16)  # fontsize of the figure title
    ax.set_xlim(0,upper_threshold)
    ax.set_ylim(0,100000)
    #ax.set_yscale('linear')
    ax.yaxis.major.formatter._useMathText = True
    ax.set_xlabel("D ($\u03BCm^2/s$)")
    #ax.set_xticklabels([0.00],['0'])
    plt.xticks([0,0.02,0.04,0.06,0.08,0.10],['0','0.02','0.04','0.06','0.08','0.10'])
    plt.yticks([0,20000,40000,60000,80000,100000])
    ax.set_ylabel("Counts")
#     plt.rcParams['axes.linewidth'] = 0.9
    #plt.tick_params(width=2)
#     ax.tick_params(width=2)
#     plt.rcParams['xtick.major.width'] = 2
#     plt.rcParams['ytick.major.width'] = 2

    #plt.legend(loc='upper right')
    #plt.xlim(0,0.15)
    if save==1:
        plt.savefig(name+'\diffusion_per_state')
    plt.show()
    
def lifetimes(state_arr, exclude_censored_lifetimes): #Calculates the lifetimes of a given state array
    bound2_lifetimes=np.array([])
    bound_lifetimes=np.array([])
    unbound_lifetimes=np.array([])
    i=0
    while i<state_arr.shape[0]-1:
        cur_state=state_arr[i]
        lifetime=0
        while cur_state==state_arr[i] and i<state_arr.shape[0]-1:
            lifetime=lifetime+1
            i=i+1
        if cur_state==0:
            unbound_lifetimes=np.append(unbound_lifetimes,lifetime)
        elif cur_state==1:
            bound_lifetimes=np.append(bound_lifetimes, lifetime)
        else:
            bound2_lifetimes=np.append(bound2_lifetimes, lifetime)
            
    if exclude_censored_lifetimes==1:
        if state_arr[0]==0:
            unbound_lifetimes=unbound_lifetimes[1:]
        elif state_arr[0]==1:
            bound_lifetimes=bound_lifetimes[1:]
        elif state_arr[0]==2:
            bound2_lifetimes=bound2_lifetimes[1:]
            
        if state_arr[state_arr.shape[0]-1]==0:
            unbound_lifetimes=unbound_lifetimes[:-1]
        elif state_arr[state_arr.shape[0]-1]==1:
            bound_lifetimes = bound_lifetimes[:-1]
        elif state_arr[state_arr.shape[0]-1]==2:
            bound2_lifetimes=bound2_lifetimes[:-1]
            
    lifetimes=[unbound_lifetimes, bound_lifetimes, bound2_lifetimes]
    return lifetimes

def lifetimes_bound(state_arr2, exclude_censored_lifetimes): #Calculates the lifetimes of a given state array
    bound2_lifetimes=np.array([])
    bound_lifetimes=np.array([])
    unbound_lifetimes=np.array([])
    i=0
    state_arr=state_arr2.copy()
    state_arr[state_arr>1]=1
    while i<state_arr.shape[0]-1:
        cur_state=state_arr[i]
        lifetime=0
        while cur_state==state_arr[i] and i<state_arr.shape[0]-1:
            lifetime=lifetime+1
            i=i+1
        if cur_state==1:
            bound_lifetimes=np.append(bound_lifetimes, lifetime)
    if exclude_censored_lifetimes==1:
        if state_arr[0]==1:
            bound_lifetimes = bound_lifetimes[1:]
            
        if state_arr[state_arr.shape[0]-1]==1:
            bound_lifetimes = bound_lifetimes[:-1]
    return bound_lifetimes

def obtain_lifetimes_from_all_states(pred_state_all, particle_index,incl_or_excl,exclude_censored_lifetimes):
    #if incl_or_excl==0:
        #print("Selected particles are INCLUDED in analysis.")
    #elif incl_or_excl==1:
        #print("Selected particles are EXCLUDED from analysis.")
    #else:
    #    print("Incl_or_excl does not has a valid input value.")
    bound2_lifetimes=np.array([])
    bound1_lifetimes=np.array([])
    unbound_lifetimes=np.array([])
    bound_lifetimes=np.array([])
    analyzed_particles = 0
    for i in range(pred_state_all.shape[0]):
        if (i in particle_index and incl_or_excl==0) or (i not in particle_index and incl_or_excl==1):
            LT = lifetimes(pred_state_all[i,:],exclude_censored_lifetimes)
            unbound_lifetimes=np.append(unbound_lifetimes,LT[0])
            bound1_lifetimes=np.append(bound1_lifetimes,LT[1])
            bound2_lifetimes=np.append(bound2_lifetimes,LT[2])
            bound_lifetimes=np.append(bound_lifetimes, lifetimes_bound(pred_state_all[i,:], exclude_censored_lifetimes))
            analyzed_particles+=1

    print('Number of analyzed particles:%1.0f' %analyzed_particles)
    return [unbound_lifetimes, bound_lifetimes, bound1_lifetimes, bound2_lifetimes]

def save_lifetimes_to_csv(unbound_LT, bound_LT, b1_LT, b2_LT, FrameRate,filepath_name):
    max_length = max(unbound_LT.shape[0], bound_LT.shape[0], b1_LT.shape[0], b2_LT.shape[0])
    unbound_list = np.zeros(max_length)
    bound_list = np.zeros(max_length)
    bound1_list = np.zeros(max_length)
    bound2_list = np.zeros(max_length)
    unbound_list[:unbound_LT.shape[0]]=unbound_LT/FrameRate
    bound_list[:bound_LT.shape[0]]=bound_LT/FrameRate
    bound1_list[:b1_LT.shape[0]]=b1_LT/FrameRate
    bound2_list[:b2_LT.shape[0]]=b2_LT/FrameRate

    d = {'unbound lifetimes': unbound_list.tolist(), 'bound lifetimes': bound_list.tolist(), 'bound1 lifetimes': bound1_list.tolist(),'bound2 lifetimes': bound2_list.tolist()}
    df = pd.DataFrame(data=d)
    df.to_csv(filepath_name + "\ " + filepath_name +'lifetimes.csv', index = False)
    print("The lifetimes are saved to .csv -> note that the 0's are supposed to be empty values and are of course not actual lifetimes.")

# Necessary functions for plotting the lifetimes.
def exp_lifetimes(x,a,b):
    output=a*np.exp(-b*x)
    return output 

def exp_lifetimes2(x,a,b,c,d):
    output=a*np.exp(-b*x)+((1-a-c)*np.exp(-d*x))
    return output 

def ecdf(sample):
    # convert sample to a numpy array, if it isn't already
    sample = np.atleast_1d(sample)

    # find the unique values and their corresponding counts
    quantiles, counts = np.unique(sample, return_counts=True)

    # take the cumulative sum of the counts and divide by the sample size to
    # get the cumulative probabilities between 0 and 1
    cumprob = np.cumsum(counts).astype(np.double) / sample.size

    return quantiles, cumprob

# Function to plot and save the lifetime -> single exponential and double exponential.
def plot_and_save_lifetime_single_expfit(lifetimes_Nframes, name, FrameRate, save, filepath_name):
    try:
        tau, f = ecdf(lifetimes_Nframes/FrameRate)
        param_bounds=([0,0],[1,np.inf])
        pred=scipy.optimize.curve_fit(exp_lifetimes,tau,1-f, np.array([0.1, 0.1]), bounds=param_bounds)
        tau_bound1 = 1/pred[0][1]
        lifetimes_fit=exp_lifetimes(tau,pred[0][0],pred[0][1])

        # save to CSV file
        d = {'time': tau, '1-f': 1-f, 'exp1_fit': exp_lifetimes(tau,pred[0][0],pred[0][1])}
        df = pd.DataFrame(data=d)
        df.to_csv(filepath_name + "\ " + filepath_name + name + ' CDF and fit.csv', index = False)    

        # plot
        fig, ax = plt.subplots(1, 1, constrained_layout=True)
        plt.rc('font', size=14)          # controls default text sizes
        plt.rc('axes', titlesize=14)     # fontsize of the axes title
        plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
        plt.rc('legend', fontsize=14)    # legend fontsize
        plt.rc('figure', titlesize=16)  # fontsize of the figure title
        ax.scatter(tau, 1-f, lw=2,color='red', s=1)
    #     ax.scatter(tau, 1-f, lw=2, label='Empirical CDF',color='red', s=1)
        ax.plot(tau, exp_lifetimes(tau,pred[0][0],pred[0][1]))
        ax.set_xlabel('Seconds')
        ax.set_ylabel('Survival (1-CDF)')
        ax.set_yscale('log')
        #ax.legend(fancybox=True, loc='right')
        ax.set_title(name +'  '+ r'$\tau$' '=  %1.3f' %tau_bound1)
        ax.set_ylim(0.01,1)

        if save==1:
            plt.savefig(filepath_name + "\ " + filepath_name + name)
        plt.show()
    except:
        tau_bound1=0
    return tau_bound1
    
def plot_and_save_lifetime_double_expfit(lifetimes_Nframes, name, FrameRate, save, filepath_name):
    try:
        tau, f = ecdf(lifetimes_Nframes/FrameRate)
        param_bounds=([0,0,0,0],[1,np.inf,1,np.inf])
        pred=scipy.optimize.curve_fit(exp_lifetimes2,tau,1-f, np.array([0.1, 0.1,0.1,0.001]), bounds=param_bounds)
        tau_1 = 1/pred[0][1]
        tau_2 = 1/pred[0][3]
        fract1 = pred[0][0]
        fract2 = 1-pred[0][0]-pred[0][2]
        lifetimes_fit=exp_lifetimes2(tau,pred[0][0],pred[0][1],pred[0][2],pred[0][3])

        # save to CSV file
        d = {'time': tau, '1-f': 1-f, 'exp2_fit': lifetimes_fit}
        df = pd.DataFrame(data=d)
        df.to_csv(filepath_name + "\ " + filepath_name + name + ' CDF and fit.csv', index = False)        

        # plot
        fig, ax = plt.subplots(1, 1, constrained_layout=True)
        plt.rc('font', size=14)          # controls default text sizes
        plt.rc('axes', titlesize=14)     # fontsize of the axes title
        plt.rc('axes', labelsize=14)    # fontsize of the x and y labels
        plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
        plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
        plt.rc('legend', fontsize=14)    # legend fontsize
        plt.rc('figure', titlesize=16)  # fontsize of the figure title
    #     ax.scatter(tau, 1-f, lw=2, label='Empirical CDF',color='red', s=1)
        ax.scatter(tau, 1-f, lw=2, color='red', s=1)
        ax.plot(tau, exp_lifetimes2(tau,pred[0][0],pred[0][1],pred[0][2],pred[0][3]))
        ax.set_xlabel('Seconds')
        ax.set_ylabel('Survival (1-CDF)')
        ax.set_yscale('log')
        #ax.legend(fancybox=True, loc='right')
        ax.set_title(name + '  ' + r'$\tau_1$' '=  %1.3f' %tau_1 +  ' , '+ r'$\tau_2$' '=  %1.3f' %tau_2 + '\n fraction1 = %1.3f' %fract1 + ', fraction2 = %1.3f' %fract2)
        ax.set_ylim(0.01,1)    
        if save==1:
            plt.savefig(filepath_name + "\ " + filepath_name + name)
        plt.show()
    except:
        tau_1=0
        tau_2=0
        fract1=0
        fract2=0
    return [tau_1, fract1, tau_2, fract2]
    
    
# Necessary functions for calculating the bound fractions.
def calculate_boundfraction_per_particle(pred_states):
    unb_fr = np.sum(pred_states==0)/pred_states.shape[0]
    b1_fr = np.sum(pred_states==1)/pred_states.shape[0]
    b2_fr = np.sum(pred_states==2)/pred_states.shape[0]
    btot_fr = b1_fr+b2_fr
    return [unb_fr, b1_fr, b2_fr, btot_fr]

def save_boundfraction_of_selected_particles(pred_state_all, particle_index, incl_or_excl, name,save):
    if incl_or_excl==0:
        #print("Selected particles are INCLUDED in analysis.")
        analyzed_particles = len(particle_index)
    elif incl_or_excl==1:
        #print("Selected particles are EXCLUDED from analysis.")
        analyzed_particles = pred_state_all.shape[0]-len(particle_index)
    else:
        print("Incl_or_excl does not has a valid input value.")
    
    particle_number = np.zeros(analyzed_particles)
    ub_fr_arr = np.zeros(analyzed_particles)
    b1_fr_arr = np.zeros(analyzed_particles)
    b2_fr_arr = np.zeros(analyzed_particles)
    btot_fr_arr = np.zeros(analyzed_particles)
    k=0
    for i in range(pred_state_all.shape[0]):
        if (i in particle_index and incl_or_excl==0) or (i not in particle_index and incl_or_excl==1):
            [unb_fr, b1_fr, b2_fr, btot_fr] = calculate_boundfraction_per_particle(pred_state_all[i,:])
            ub_fr_arr[k] = unb_fr
            b1_fr_arr[k] = b1_fr
            b2_fr_arr[k] = b2_fr
            btot_fr_arr[k] = btot_fr
            particle_number[k] = i 
            k+=1
            
    part_num_list=particle_number.tolist()
    part_num_list.insert(0,'Total')
    unb_fr_list = ub_fr_arr.tolist()
    unb_fr_list.insert(0,np.mean(ub_fr_arr))
    b1_fr_list=b1_fr_arr.tolist()
    b1_fr_list.insert(0,np.mean(b1_fr_arr))
    b2_fr_list=b2_fr_arr.tolist()
    b2_fr_list.insert(0,np.mean(b2_fr_arr))
    btot_fr_list=btot_fr_arr.tolist()
    btot_fr_list.insert(0,np.mean(btot_fr_arr))
    
    if save==1:
        d = {'particle number': part_num_list, 'unbound fraction': unb_fr_list, 'single bound fraction': b1_fr_list, 'double bound fraction': b2_fr_list, 'bound fraction': btot_fr_list}
        df = pd.DataFrame(data=d)
        df.to_csv(name+"\ "+'boundfractions.csv', index = False)

    print('Number of analyzed particles:%1.0f' %analyzed_particles)
    print("The total unbound/single bound/double bound fractions of the selected particles is equal to %1.3f" %np.mean(ub_fr_arr)+ "/%1.3f" %np.mean(b1_fr_arr) + "/%1.3f" %np.mean(b2_fr_arr))
    return [np.mean(ub_fr_arr),np.mean(b1_fr_arr),np.mean(b2_fr_arr)]

# Necessary functions for calculating the activity of specific particles.
def calculate_activity_per_particle(state_arr):
    activity01=0
    activity12=0
    activitytotal=0
    i=0
    prev_state=state_arr[i]
    while i<state_arr.shape[0]-1:
        cur_state=state_arr[i]
        if cur_state!=prev_state:
            if prev_state==0:
                if cur_state==1:
                    activity01+=1
                    activitytotal+=1
                elif cur_state==2:
                    activity01+=1
                    activity12+=1
                    activitytotal+=2
            elif prev_state==1:
                if cur_state==0:
                    activity01+=1
                    activitytotal+=1
                elif cur_state==2:
                    activity12+=1
                    activitytotal+=1
            elif prev_state==2:
                if cur_state==0:
                    activity01+=1
                    activity12+=1
                    activitytotal+=2
                elif cur_state==1:
                    activity12+=1
                    activitytotal+=1
        prev_state=cur_state
        i+=1
    return [activity01, activity12, activitytotal]
    

def save_activity_of_selected_particles(pred_state_all, particle_index, incl_or_excl,N_frames,FrameRate,name,save):
    if incl_or_excl==0:
        #print("Selected particles are INCLUDED in analysis.")
        analyzed_particles = len(particle_index)
    elif incl_or_excl==1:
        #print("Selected particles are EXCLUDED from analysis.")
        analyzed_particles = pred_state_all.shape[0]-len(particle_index)
    else:
        print("Incl_or_excl does not have a valid input value.")
        
    particle_number = np.zeros(analyzed_particles)
    act01 = np.zeros(analyzed_particles)
    act12 = np.zeros(analyzed_particles)
    acttot = np.zeros(analyzed_particles)
    k=0
    for i in range(pred_state_all.shape[0]):
        if (i in particle_index and incl_or_excl==0) or (i not in particle_index and incl_or_excl==1):
            [activity01, activity12, activitytotal] = calculate_activity_per_particle(pred_state_all[i,:])
            act01[k] = activity01
            act12[k] = activity12
            acttot[k] = activitytotal
            particle_number[k] = i 
            k+=1
            
    part_num_list=particle_number.tolist()
    part_num_list.insert(0,'Mean Activity')
    part_num_list.insert(0,'Total Activity')
    part_num_list.insert(0,'Activity mHz')
    
    act01_list = act01.tolist()
    act01_list.insert(0,np.mean(act01))
    act01_list.insert(0,np.sum(act01))
    act01_list.insert(0,np.mean(act01)/(np.floor(N_frames/FrameRate))*1000)
    
    act12_list = act12.tolist() 
    act12_list.insert(0,np.mean(act12))
    act12_list.insert(0,np.sum(act12))
    act12_list.insert(0,np.mean(act12)/(np.floor(N_frames/FrameRate))*1000)
    
    acttot_list=acttot.tolist()
    acttot_list.insert(0,np.mean(acttot))
    acttot_list.insert(0,np.sum(acttot))
    acttot_list.insert(0,np.mean(acttot)/(np.floor(N_frames/FrameRate))*1000)
    
    if save==1:
        d = {'particle number': part_num_list, '01 Activity': act01_list, '12 Activity': act12_list, 'Total Activity': acttot_list}
        df = pd.DataFrame(data=d)
        df.to_csv(name+"\ "+'activity.csv', index = False)

    print('Number of analyzed particles:%1.0f' %analyzed_particles)
    print("The total and mean 01 activity selected particles is equal to %1.3f" %np.sum(act01)+ "/%1.3f" %np.mean(act01))
    print("The total and mean 12 activity selected particles is equal to %1.3f" %np.sum(act12)+ "/%1.3f" %np.mean(act12))
    print("The total and mean activity selected particles is equal to %1.3f" %np.sum(acttot)+ "/%1.3f" %np.mean(acttot))
    return [np.mean(act01), np.mean(act12), np.mean(acttot)]

### Function to do the whole analysis at once.

In [3]:
# Function that obtains all the .txt files in the directory.
def obtain_txt_file_names_in_dir():
    namelist = []
    for filename in os.listdir():
        if filename.endswith(".txt"):
            namelist.append(os.path.splitext(filename)[0])
    return namelist

# Function that performs the whole deep learning analysis.
def perform_deep_learning_analysis(PixelSize, FrameRate, model01, model12, name, measurementwindow1, measurementwindow2, MVbound_threshold, sigma, distribution_fit, exclude_censored_lifetimes):
    os.mkdir(name)
    print(name)
    data = np.genfromtxt(name+".txt", delimiter=',')
    # Scale the coordinates with the pixelsize.
    xy_list = data[:,:-1]*PixelSize
    num_particles = data.shape[0]
    print('The total number of particles is equal to %1.0f' %num_particles)

    # Obtain xy coordinates for each particle
    x_per_part=obtain_xy_per_particle(xy_list)[0]
    y_per_part=obtain_xy_per_particle(xy_list)[1]
    N_frames = x_per_part.shape[1]
    print('The number of frames per particle is equal to %1.0f' %N_frames + ', which corresponds to %1.0f seconds.' %np.floor(N_frames/FrameRate))

    # Function to downscale the FrameRate from 60 to 30 -> this allows the deep learning algorithm to work faster (at similar performance)
    # If this function is used -> also need to use the deep learning model for 30 FrameRate, rather than the one for 60 FrameRate. (furthermore, the variable FrameRate is automatically set to 30)
    if FrameRate==60:
        [x_per_part, y_per_part, N_frames, FrameRate] = downscale_FR60_to_FR30(x_per_part, y_per_part)

    # Empty array to store the predicted states of all the particles.
    pred_state_all = np.zeros((num_particles,N_frames))

    no_MSD_val=10 #number of MSD values used for the diffusion coefficient calculation -> called maxdt in the matlab script.
    N_val_DiffCoef = 100 #measurementwindow for the diffusion coefficient calculation. 

    diff_coef_all = np.zeros((num_particles, N_frames-N_val_DiffCoef+1)) # Empty array to store the calculated values of the diffusion coefficient. 
    Time = np.linspace(1/FrameRate, N_frames/FrameRate, N_frames)

    #Normalization of the x,y values. Important for training deep learning model (goes faster when input values are of order unity). Do not change unless specified (currently not the case). 
    #Currently, the simulation is outputted in meters (norm4e-7) while the experimental data is outputted in um (therefore norm 0.4)
    norm=0.4 
    norm2=norm

    smooth01 = measurementwindow+1
    smooth12 = measurementwindow2+1
    print('xy-scatter plot, blue=unbound, green=single bound, red=double bound. Diffusion coefficient plot is added next to the predicted states for reference, it is not used to determine the states.')
    for p_num in range(num_particles):
    #for p_num in range(5):
        x=x_per_part[p_num,:]
        y=y_per_part[p_num,:]

        X_test = X_test_from_timeseries(x,y,measurementwindow) # Create input subtrajectories for classification. 
        for i in range(X_test.shape[0]):
            X_test[i,:,:2]-=X_test[i,0,:2] # minus the first coordinate to start the trajectories in the same spot.
        X_test=X_test/norm #normalize by the bound radius of the first state.
    
        # Obtain predictions for unbound/bound
        Ypred01_smooth=smoother_unev(obtain_state_per_pos(model01.predict(X_test), measurementwindow),smooth01)
        # Obtain predictions for single/double bond
        Ypred12_smooth = obtain_pred_state2(x, y, Ypred01_smooth, measurementwindow2, norm2, model12, smooth12)
        print(len(Ypred12_smooth))
        #Save the predicted states for later analysis.
        pred_state_all[p_num,:] = Ypred12_smooth
        
        diff_coef_all[p_num,:] = diff_coef_over_time(x,y,N_val_DiffCoef,no_MSD_val, FrameRate)

        
        ## Plot 
        if p_num<50:
            Ypred12_smooth_plot = pred_state_all[p_num,0:len(diff_coef_all[0])]
            Time_plot = Time[0:len(diff_coef_all[0])]
            x=x_per_part[p_num,0:len(diff_coef_all[0])]
            y=y_per_part[p_num,0:len(diff_coef_all[0])]
            
            SMALL_SIZE = 24
            MEDIUM_SIZE = 26
            BIGGER_SIZE = 28

            ubtime=Time_plot.copy()
            sbtime=Time_plot.copy()
            mbtime=Time_plot.copy()
            ubdiffcoeff=diff_coef_all.copy()
            sbdiffcoeff=diff_coef_all.copy()
            mbdiffcoeff=diff_coef_all.copy()
            ubstate=Ypred12_smooth_plot.copy()
            sbstate=Ypred12_smooth_plot.copy()
            mbstate=Ypred12_smooth_plot.copy()

            for timepoint1 in range(len(Ypred12_smooth_plot)):
                if Ypred12_smooth_plot[timepoint1] != 0:
                    ubtime[timepoint1]= np.nan
                    ubdiffcoeff[p_num,timepoint1]=np.nan
                    ubstate[timepoint1]=np.nan
            #         display(timepoint1)

            for timepoint2 in range(len(Ypred12_smooth_plot)-1):
                if Ypred12_smooth_plot[timepoint2] != 1:
                    sbtime[timepoint2]= np.nan
                    sbdiffcoeff[p_num,timepoint2]=np.nan
                    sbstate[timepoint1]=np.nan

            for timepoint3 in range(len(Ypred12_smooth_plot)-1):
                if Ypred12_smooth_plot[timepoint3] != 2:
                    mbtime[timepoint3]= np.nan
                    mbdiffcoeff[p_num,timepoint3]=np.nan
                    mbstate[timepoint1]=np.nan
            
            particlenumber=p_num+1
            
            print(name + ': plots for particle %1.0f' %particlenumber)
#             fig2, axs = plt.subplots(1, 2, sharex=False, sharey=False, figsize=(24,8), gridspec_kw={'width_ratios': [1, 2]})
#             axs[0].scatter(x[Ypred12_smooth==0], y[Ypred12_smooth==0], s=1)
#             axs[0].scatter(x[Ypred12_smooth==1], y[Ypred12_smooth==1], s=2, color='green')
#             axs[0].scatter(x[Ypred12_smooth==2], y[Ypred12_smooth==2], s=0.2, color='red')

#             axs[1].plot(Time[np.floor(N_val_DiffCoef/2).astype(int)-1:-np.floor(N_val_DiffCoef/2).astype(int)], diff_coef_all[p_num,:])
#             axs[1].plot(Time, -1*Ypred12_smooth/20)

            fig2, axs = plt.subplot_mosaic([['xy', 'D'], ['xy', 'state']],constrained_layout=True,  figsize=(24,8), gridspec_kw={'width_ratios': [1, 2]})
            axs['xy'].scatter(x[Ypred12_smooth_plot==0], y[Ypred12_smooth_plot==0], s=1)
            axs['xy'].scatter(x[Ypred12_smooth_plot==1], y[Ypred12_smooth_plot==1], s=2, color='green')
            axs['xy'].scatter(x[Ypred12_smooth_plot==2], y[Ypred12_smooth_plot==2], s=0.2, color='red')
            axs['xy'].set_xlabel("x-coordinate (\u03BCm)")
            axs['xy'].set_ylabel("y-coordinate (\u03BCm)")
            plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
            plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
            plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
            plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
            plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
            plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

            axs['D'].plot(ubtime, ubdiffcoeff[p_num,:])
            axs['D'].plot(sbtime, sbdiffcoeff[p_num,:], color='green')
            axs['D'].plot(mbtime, mbdiffcoeff[p_num,:], color='red')
            #axs['D'].set_xlabel("Time (s)")
            axs['D'].set_ylabel("D ($\u03BCm^2/s$)")
            axs['D'].set_ylim([0,0.1])
            axs['D'].set_xlim([0,np.floor(N_frames/FrameRate)])
            axs['D'].set_yticks(np.arange(0,0.11,step=0.02))
            
            axs['state'].plot(Time_plot, -1*Ypred12_smooth_plot, 'k--',linewidth=2)
            axs['state'].plot(ubtime, -1*ubstate,linewidth=4)
            axs['state'].plot(sbtime, -1*sbstate, color='green', linewidth=4)
            axs['state'].plot(mbtime, -1*mbstate, color='red',linewidth=4)
            plt.yticks([0,-1,-2],['Unbound','Single bound','Double bound'])
            axs['state'].set_xlabel("Time (s)")
            axs['state'].set_xlim([0,np.floor(N_frames/FrameRate)])
            #axs['state'].set_ylabel("State")
            
            fig2.savefig(name + "\ " + name + 'Particle %1.0f' %p_num)

            plt.show()
            #plt.plot(Time,x-x[0])
            #plt.plot(Time,y-y[0])
            #plt.show()
        
 
    # Save results into specific folder
    pd.DataFrame(pred_state_all).to_csv(name+"\predicted_state_all.csv")
    pd.DataFrame(diff_coef_all).to_csv(name+"\Dtotal.csv")
    # Remove stuck particles.
    stuck_particle_ind = find_stuck_particles(x_per_part, y_per_part, FrameRate, MVbound_threshold)
    print('The total number of stuck particles is equal to: %1.0f' %stuck_particle_ind.shape[0])

    # Remove/Recognize mislocalized particles
    misl_part_index = find_mislocalizations(x_per_part, y_per_part, FrameRate).astype(int)
    print('The total number of mislocalized particles is equal to: %1.0f' %misl_part_index.shape[0])

    # Filter the particles that have a large activity in total and particles that have an unusually large activity between the 01 state.
    [act01, act12, acttot] = calc_activity_of_particles(pred_state_all)
    filter01 = filter_large_activity(act01, stuck_particle_ind,distribution_fit,sigma)
    # Filter the particles with high 12 activity -> but do not include particles that are only freely diffusing -> otherwise the algorithm will throw out almost all the particles with 12 activity.
    # freely_diff_part = find_freely_diffusing_particles(act01, x_per_part, y_per_part, FrameRate)

    # Filter the particles with high 12 activity -> but do not include particles that show no 12 activity at all ->otherwise the algorithm will throw out almost all the particles with any 12 activity.
    freely_diff_part = np.where(act12==0)[0]
    filter12 = filter_large_activity(act12, np.append(stuck_particle_ind, freely_diff_part),distribution_fit,sigma+3)

    filtered_particles = np.unique(np.append(filter01, filter12))
    print('The total number of filtered particles is equal to: %1.0f' %filtered_particles.shape[0])

    remove_particle_all = np.unique(np.append(np.append(stuck_particle_ind, misl_part_index), filtered_particles))
    
    # Plot and save distributions etc.
    save=1
    particle_index= remove_particle_all #exclude the particles that are filtered out above.
    # Indicate whether you want to include/exclude the indicated particles. To include: incl_or_excl=0. To exclude: incl_or_excl=1.
    incl_or_excl=1
    upper_threshold = 0.1 # do not plot values of the diffusion coefficient larger than this value.

    #Plot the complete diffusion coefficient distribution
    plot_diff_coef_hist(diff_coef_all, particle_index, incl_or_excl,upper_threshold,save,name)

    #Plot the diffusion coefficient distributions of unbound, single and double bound seperately.
    plot_diff_coef_hist_per_pred_state(diff_coef_all, pred_state_all, particle_index, incl_or_excl, N_val_DiffCoef, upper_threshold,save,name)
    
    # Determine Lifetimes per particle state trace
    [unbound_lifetimes_Nframes, bound_lifetimes_Nframes, bound1_lifetimes_Nframes, bound2_lifetimes_Nframes] = obtain_lifetimes_from_all_states(pred_state_all, particle_index, incl_or_excl,exclude_censored_lifetimes)
    
    tau_unb_single = plot_and_save_lifetime_single_expfit(unbound_lifetimes_Nframes, 'unbound lifetime exp1', FrameRate,save,name)
    [tau_1_unb, fract1_unb, tau_2_unb, fract2_unb] = plot_and_save_lifetime_double_expfit(unbound_lifetimes_Nframes, 'unbound lifetime exp2', FrameRate, save,name)
    [tau_1_bound, fract1_bound, tau_2_bound, fract2_bound] = plot_and_save_lifetime_double_expfit(bound_lifetimes_Nframes, 'all bound lifetimes exp2', FrameRate,save,name)
    tau_b1_single = plot_and_save_lifetime_single_expfit(bound1_lifetimes_Nframes, 'single bound lifetime exp1', FrameRate,save,name)
    [tau_1_b1, fract1_b1, tau_2_b1, fract2_b1] = plot_and_save_lifetime_double_expfit(bound1_lifetimes_Nframes, 'single bound lifetime exp2', FrameRate,save,name)
    tau_1_b2_single = plot_and_save_lifetime_single_expfit(bound2_lifetimes_Nframes, 'double bound lifetime exp1', FrameRate,save,name)
    [tau_1_b2, fract1_b2, tau_2_b2, fract2_b2] = plot_and_save_lifetime_double_expfit(bound2_lifetimes_Nframes, 'double bound lifetime exp2', FrameRate,save,name)
    
    # Determine bound fraction and activity.
    [mean_bf01, mean_bf12, mean_bftot] = save_boundfraction_of_selected_particles(pred_state_all, particle_index, incl_or_excl, name,1)
    [mean_act01, mean_act12, mean_acttot] = save_activity_of_selected_particles(pred_state_all, particle_index, incl_or_excl,N_frames,FrameRate, name,1)
    
    # Functions to determine unfiltered bound fraction/activity/lifetimes of non-filtered vs. filtered. 
    [mean_bf01_unf, mean_bf12_unf, mean_bftot_unf] = save_boundfraction_of_selected_particles(pred_state_all, [], 0, name,0)
    [mean_act01_unf, mean_act12_unf, mean_acttot_unf] = save_activity_of_selected_particles(pred_state_all, [], 0, N_frames, FrameRate, name,0)
    
    # Determine Lifetimes per particle state trace - unfiltered.
    [unbound_lifetimes_Nframes, bound_lifetimes_Nframes, bound1_lifetimes_Nframes, bound2_lifetimes_Nframes] = obtain_lifetimes_from_all_states(pred_state_all, [], 1,exclude_censored_lifetimes)
    ## Save lifetimes (in seconds) of specified particles to a csv file -> UNCOMMENT TO USE.
    save_lifetimes_to_csv(unbound_lifetimes_Nframes, bound_lifetimes_Nframes, bound1_lifetimes_Nframes, bound2_lifetimes_Nframes, FrameRate, name)
    
    tau_unb_single_unf = plot_and_save_lifetime_single_expfit(unbound_lifetimes_Nframes, 'unbound lifetime exp1 - unfiltered', FrameRate,save,name)
    [tau_1_unb_unf, fract1_unb_unf, tau_2_unb_unf, fract2_unb_unf] = plot_and_save_lifetime_double_expfit(unbound_lifetimes_Nframes, 'unbound lifetime exp2 - unfiltered', FrameRate, save,name)
    [tau_1_bound_unf, fract1_bound_unf, tau_2_bound_unf, fract2_bound_unf] = plot_and_save_lifetime_double_expfit(bound_lifetimes_Nframes, 'bound lifetime exp2 - unfiltered', FrameRate,save,name)
    tau_b1_single_unf = plot_and_save_lifetime_single_expfit(bound1_lifetimes_Nframes, 'single bound lifetime exp1- unfiltered', FrameRate,save,name)
    [tau_1_b1_unf, fract1_b1_unf, tau_2_b1_unf, fract2_b1_unf] = plot_and_save_lifetime_double_expfit(bound1_lifetimes_Nframes, 'single bound lifetime exp2 - unfiltered', FrameRate,save,name)
    tau_1_b2_single_unf = plot_and_save_lifetime_single_expfit(bound2_lifetimes_Nframes, 'double bound lifetime exp1 - unfiltered', FrameRate,save,name)
    [tau_1_b2_unf, fract1_b2_unf, tau_2_b2_unf, fract2_b2_unf] = plot_and_save_lifetime_double_expfit(bound2_lifetimes_Nframes, 'double bound lifetime exp2 - unfiltered', FrameRate,save,name)
    

    return [mean_bf01, mean_bf12, mean_bftot, mean_act01, mean_act12, mean_acttot, mean_bf01_unf, mean_bf12_unf, mean_bftot_unf, mean_act01_unf, mean_act12_unf, mean_acttot_unf, tau_unb_single, tau_1_unb, fract1_unb, tau_2_unb, fract2_unb, tau_1_bound, fract1_bound, tau_2_bound, fract2_bound, tau_b1_single, tau_1_b1, fract1_b1, tau_2_b1, fract2_b1, tau_1_b2_single, tau_1_b2, fract1_b2, tau_2_b2, fract2_b2, tau_unb_single_unf, tau_1_unb_unf, fract1_unb_unf, tau_2_unb_unf, fract2_unb_unf, tau_1_bound_unf, fract1_bound_unf, tau_2_bound_unf, fract2_bound_unf, tau_b1_single_unf, tau_1_b1_unf, fract1_b1_unf, tau_2_b1_unf, fract2_b1_unf, tau_1_b2_single_unf, tau_1_b2_unf, fract1_b2_unf, tau_2_b2, fract2_b2_unf]

## Data Import and Model selection

### Important note #1: the PixelSize needs to be accurately filled in for each analyzed dataset.

The deep learning algorithm has been trained on specific motion patterns with a certain bound radius and relative changes in x and y. Using the wrong pixelsize results in smaller/larger bound radius and smaller/larger changes in x and y coordinates. Therefore, if wrong, the algorithm will classify the trajectory more as (double) bound/unbound respectively.

### Important note #2
The deep learning models have been trained on a framerate of 30.  Therefore, datasets that are measured at 60 frames/s are downscaled to 30/s automatically. 

Furthermore, the measurementwindow of the deep learning model must be saved in python as a parameter. The measurementwindow of the deep learning model is specified in the title. 

    Measurementwindow -> corresponds to the measurementwindow of model01 - the model to differentiate between unbound/bound.
    Measurementwindow2 -> corresponds to the measurementwindow of model12 - the model to differentiate between single/double bound.

The measurementwindow varies for 1 um and 3 um particles -> so check if its correctly filled in!!

In [4]:
PixelSize=0.345; #Leica M1/M3 with grasshopper3 = 0.345 um/pixel; M2 = 0,588; correct for c-mount x1 
FrameRate=60 # Fill in the framerate of the measurement. (not the framerate of the deep learning model)

# Model to differentiate between unbound/bound.
model01 = keras.models.load_model('DLmodel01_3um_meas120_norm4e-7_FR30.h5')
# Model to differentiate between single/double bound.
model12 = keras.models.load_model('DLmodel12_3um_meas180_norm4e-7_FR30.h5')
measurementwindow = 120
measurementwindow2 = 180

### Settings for automatic filtering of particles
Firstly, the stuck particles are removed. This is done by setting a threshold on the mean diffusion coefficient of a particle. If that value is too low, the particle is classified as stuck.

Furthermore, mislocalized particles are removed. These are particles which move close to ecah other & afterwards are seen as the same particle in the localization software.

Lastly, the particles with an unusually large activity are removed. Namely, in the experiment, often certain particles have a much higher activity compared to the whole, these are actvity outliers and are removed. This removal is done via the same filtering method Alissa uses in her matlab script.

In [6]:
# Remove stuck particles.
MVbound_threshold = 0.005 # Threshold for the mean diffusion coefficient which determines wether a particle is classified as 'stuck'.

# Filtering outliers
# Indicate the sigma value you want to use for the standard deviation -> sigma indicates the number of standard deviations a particle must surpass for it to be classified as 'unusually large number of events'.
sigma=4
# Indicate which fittype you want to use for the filtering by uncommenting the desired distribution.
distribution_fit = 'poisson'
#distribution_fit = 'normal'

# Indicate whether you want to exclude censored lifetimes. To exclude: exclude_censored_lifetimes=1.
exclude_censored_lifetimes=0
# Censored lifetimes: lifetimes that 'begin' or 'end' at the start or end of the measurement respectively, and therefore are not completely observed.

## Start Analyzing All .txt Files
Code to loop through all the .txt files.

Common error code: if there is already a folder with the same name as the .txt file -> it will produce an error.

In [ ]:
all_txt_filenames = obtain_txt_file_names_in_dir()
# Loop through all the .txt files in the directory.
N_txt_files = len(all_txt_filenames)
value_mat = np.zeros((N_txt_files, 50))
k=0
for name in all_txt_filenames:
    # perform_deep_learning_analysis(PixelSize, FrameRate, model01, model12, name, measurementwindow, measurementwindow2, MVbound_threshold, sigma, distribution_fit, exclude_censored_lifetimes)
    # 30 values as output -> save in excel file.
    values_out = perform_deep_learning_analysis(PixelSize, FrameRate, model01, model12, name, measurementwindow, measurementwindow2, MVbound_threshold, sigma, distribution_fit, exclude_censored_lifetimes)
    value_mat[k,:] = np.array(values_out)
    k+=1
    
# Save the found values for all the txt files in a single .csv file.
for i in range(value_mat.shape[1]):
    list_i = value_mat[:,i].tolist()
    exec(f'csv_{i+1} = list_i')

d = {'name txt file': all_txt_filenames, 'Mean Unbound Fraction Filtered': csv_1, 'Mean b1 Fraction Filtered': csv_2, 'Mean b2 Fraction Filtered': csv_3, 'Mean 01 Activity Filtered': csv_4, 'Mean 12 Activity Filtered': csv_5, 'Mean Total Activity Filtered': csv_6, 'Mean Unbound Fraction Unfiltered': csv_7, 'Mean b1 Fraction Unfiltered': csv_8, 'Mean b2 Fraction Unfiltered': csv_9, 'Mean 01 Activity Unfiltered': csv_10, 'Mean 12 Activity Unfiltered': csv_11, 'Mean Total Activity Unfiltered': csv_12, 'Unbound Lifetime Filtered - Single exp. fit': csv_13, 'Unbound Lifetime Filtered - Double exp. fit - tau1': csv_14, 'Unbound Lifetime Filtered - Double exp. fit - fraction1': csv_15, 'Unbound Lifetime Filtered - Double exp. fit - tau2': csv_16, 'Unbound Lifetime Filtered - Double exp. fit - fraction2': csv_17, 'Bound Lifetime Filtered - Double exp. fit - tau1': csv_18, 'Bound Lifetime Filtered - Double exp. fit - fraction1': csv_19, 'Bound Lifetime Filtered - Double exp. fit - tau2': csv_20, 'Bound Lifetime Filtered - Double exp. fit - fraction2': csv_21, 'Single Bound Lifetime Filtered - Single exp. fit': csv_22, 'Single Bound Lifetime Filtered - Double exp. fit - tau1': csv_23, 'Single Bound Lifetime Filtered - Double exp. fit - fraction1': csv_24, 'Single Bound Lifetime Filtered - Double exp. fit - tau2': csv_25, 'Single Bound Lifetime Filtered - Double exp. fit - fraction2': csv_26,'Double Bound Lifetime Filtered - Single exp. fit': csv_27, 'Double Bound Lifetime Filtered - Double exp. fit - tau1': csv_28, 'Double Bound Lifetime Filtered - Double exp. fit - fraction1': csv_29, 'Double Bound Lifetime Filtered - Double exp. fit - tau2': csv_30, 'Double Bound Lifetime Filtered - Double exp. fit - fraction2': csv_31, 'Unbound Lifetime Unfiltered - Single exp. fit': csv_32, 'Unbound Lifetime Unfiltered - Double exp. fit - tau1': csv_33, 'Unbound Lifetime Unfiltered - Double exp. fit - fraction1': csv_34, 'Unbound Lifetime Unfiltered - Double exp. fit - tau2': csv_35, 'Unbound Lifetime Unfiltered - Double exp. fit - fraction2': csv_36, 'Bound Lifetime Unfiltered - Double exp. fit - tau1': csv_37, 'Bound Lifetime Unfiltered - Double exp. fit - fraction1': csv_38, 'Bound Lifetime Unfiltered - Double exp. fit - tau2': csv_39, 'Bound Lifetime Unfiltered - Double exp. fit - fraction2': csv_40, 'Single Bound Lifetime Unfiltered - Single exp. fit': csv_41, 'Single Bound Lifetime Unfiltered - Double exp. fit - tau1': csv_42, 'Single Bound Lifetime Unfiltered - Double exp. fit - fraction1': csv_43, 'Single Bound Lifetime Unfiltered - Double exp. fit - tau2': csv_44, 'Single Bound Lifetime Unfiltered - Double exp. fit - fraction2': csv_45,'Double Bound Lifetime Unfiltered - Single exp. fit': csv_46, 'Double Bound Lifetime Unfiltered - Double exp. fit - tau1': csv_47, 'Double Bound Lifetime Unfiltered - Double exp. fit - fraction1': csv_48, 'Double Bound Lifetime Unfiltered - Double exp. fit - tau2': csv_49, 'Double Bound Lifetime Unfiltered - Double exp. fit - fraction2': csv_50}
df = pd.DataFrame(data=d)
df.to_csv('output_all.csv', index = False)